### Target Encodingとは

Target Encoding（Target Mean Encoding）とはカテゴリカル（質的）データを数値に変換する方法の1つです。

様々な手法があるのですが、Target Encodingの一番の特徴は目的変数を使用するという点です。筆者の言葉で誤解を恐れずに言うのであればTarget Encodingが生み出すのは「値が大きいほど目的変数の値も大きい確率が高い」特徴量ということになります。

目的変数という答えを利用するTarget Encodingはデータセットによっては非常に強力な力を持ちます。

Target Encodingの変換を言葉にすると「目的変数の平均値を特徴量にする」です。

外れ値などの状況によっては中央値などをとる場合もあるのですが、本稿では平均値を使用します。

イメージとしては図1のようになります。本稿では可能な限り理解しやすいよう、0と1の2値分類を想定しています。

![](./img/image1.png)

Target Encodingの基本はこれだけです。図1の場合だと、特徴量Aは目的変数が1をとる確率が他に比べて高いということになります。

ここまでは難しいと感じない方がほとんどだと思います。

しかしTarget Encodingが難しいと言われる所以は、この「訓練データの目的変数の平均」をどのようにして取得するかという点にあります。

取得方法については後述します

#### リークとは？

#### テストデータへの適応

### Target Encodingの種類

先ほど少し触れましたが、Target Encodingには「目的変数の平均」の取り方によっていくつか種類があります。リークを起こしやすいものから起こしにくいものまで存在するので、しっかりと理解して扱えるようにする必要があります。本節では仮のデータセットを作成し、ある程度わかりやすい数値でTarget Encodingを実装していきます。解説では数値がどのように変化しているのか1つ1つ追っていきます。本稿で解説するのは以下の3つです。

- Greedy Target Encoding
- Leave-one-out Target Encoding
- Holdout Targer Encoding

### Greedy Target Encoding

最初は、Greedy Target Encoding（Greedy Target Statistic）です。

この方法は単純にデータセット全体から目的変数の平均を取得します。

しかし、この方法はリークが起こる可能性があります。図2は特徴量の1つに焦点を当てた時のGreedy Target Encodingです。

リークが起こる際の理由に関しては実装コードを確認した上で解説します。

![](./img/image2.png)


まずは仮のデータセットを用意します。

pandsをインポートし、DataFrameでデータセットを作成します。

仮のデータセットは10行×3列です。

目的変数以外の2列に関しては文字列で作成します。

このDataFrameを用いて3種類のTarget Encodingの実装を行います。

In [69]:
#ライブラリのインポート
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
 
#DataFrameで仮の訓練データセットを作成（10×3）
df = pd.DataFrame({
    "column1" : ["A","A","A","A","A","B","B","B","B","B"],
    "column2" : ["C","D","D","D","D","D","D","D","E","E"],
    "target" : [1, 0, 0, 1, 0, 0, 1, 1, 0, 1]
})
 
#先頭から10行を表示
df.head(10)
 

,column1,column2,target
0,A,C,1
1,A,D,0
2,A,D,0
3,A,D,1
4,A,D,0
5,B,D,0
6,B,D,1
7,B,D,1
8,B,E,0
9,B,E,1


仮のデータセットが作成できました。

targetの列が今回の目的変数となります。

column1とcolumn2は文字列ですので、数値に変換してあげる必要があります。

そこで先ほど解説したGreedy Target Encodingを用いてcolumn1とcolumn2を数値に変換します。

まずは各特徴量ごとの目的変数の平均をとります。column1であればAとB、column2であればCとDとEです。

In [70]:
col1_means = df.groupby('column1')['target'].mean()
col2_means = df.groupby('column2')['target'].mean()

In [71]:
print(col1_means)
print(col2_means)

column1
A    0.4
B    0.6
Name: target, dtype: float64
column2
C    1.000000
D    0.428571
E    0.500000
Name: target, dtype: float64


In [72]:
df["column1_target"] = df["column1"].map(col1_means)
df["column2_target"] = df["column2"].map(col2_means)
df

,column1,column2,target,column1_target,column2_target
0,A,C,1,0.4,1.000000
1,A,D,0,0.4,0.428571
2,A,D,0,0.4,0.428571
3,A,D,1,0.4,0.428571
4,A,D,0,0.4,0.428571
5,B,D,0,0.6,0.428571
6,B,D,1,0.6,0.428571
7,B,D,1,0.6,0.428571
8,B,E,0,0.6,0.500000
9,B,E,1,0.6,0.500000


Greedy Target Encodingが完了しました。

簡単に実装できましたが、今回の実装ではリークが起こっています。

1行目を見てください。目的変数（target）と特徴量（column2_target）の値が同じです。

Greedy Target Encodingでは自分自身の目的変数も計算に加えてしまっているため、カテゴリカル変数の数が1つだと特徴量の値がそのまま目的変数になっています。

これではテストデータが与えられた時に正しく評価することは厳しく、正しい特徴量生成とは言えません。

場合によってはSmoothingなどのテクニックを使ってリークを起こしにくくすることもできます。Smoothingに関しては後述します。

### Leave-one-out Target Encoding

次に、Leave-one-out Target Encoding（Leave-one-out Target Statistic）です。

この方法は自身が持つ目的変数以外の目的変数の平均を取得します。

この方法もリークが起こる可能性があるので注意が必要です。

図3は特徴量の1つに焦点を当てた時のLeave-one-out Target Encodingです。

こちらも、リークが起こる際の理由に関しては、実装コードを確認した上で解説します。

![](./img/image3.png)

In [73]:
#ライブラリのインポート
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
 
#DataFrameで仮の訓練データセットを作成（10×3）
df = pd.DataFrame({
    "column1" : ["A","A","A","A","A","B","B","B","B","B"],
    "column2" : ["C","D","D","D","D","D","D","D","E","E"],
    "target" : [1, 0, 0, 1, 0, 0, 1, 1, 0, 1]
})
 
#先頭から10行を表示
df.head(10)

,column1,column2,target
0,A,C,1
1,A,D,0
2,A,D,0
3,A,D,1
4,A,D,0
5,B,D,0
6,B,D,1
7,B,D,1
8,B,E,0
9,B,E,1


Greedy Target Encodingでは目的変数の平均を取得するだけで実装できましたが、Leave-one-out Target Encodingでは自身の値は含まないため、実装が少し複雑になります。

まずは特徴毎のカウント数と目的変数の合計を取得します。

In [74]:
# まず集計
col1_sum = df.groupby('column1')['target'].sum()
col1_count = df.groupby('column1')['target'].count()

col2_sum = df.groupby('column2')['target'].sum()
col2_count = df.groupby('column2')['target'].count()

# 各行に対応するカウントと合計をマージ（ベクトル演算）
df['col1_sum'] = df['column1'].map(col1_sum)
df['col1_count'] = df['column1'].map(col1_count)

df['col2_sum'] = df['column2'].map(col2_sum)
df['col2_count'] = df['column2'].map(col2_count)

# Leave-One-Out Target Encodingを計算
# 分母0回避のため、count > 1の場合のみLOO計算、それ以外は全体平均にする
global_mean = df['target'].mean()

# column1用
df['column1_target'] = ((df['col1_sum'] - df['target']) / (df['col1_count'] - 1)).where(df['col1_count'] > 1, global_mean)

# column2用
df['column2_target'] = ((df['col2_sum'] - df['target']) / (df['col2_count'] - 1)).where(df['col2_count'] > 1, global_mean)

# 不要な中間列を削除
df.drop(['col1_sum', 'col1_count', 'col2_sum', 'col2_count'], axis=1, inplace=True)

df

,column1,column2,target,column1_target,column2_target
0,A,C,1,0.25,0.500000
1,A,D,0,0.50,0.500000
2,A,D,0,0.50,0.500000
3,A,D,1,0.25,0.333333
4,A,D,0,0.50,0.500000
5,B,D,0,0.75,0.500000
6,B,D,1,0.50,0.333333
7,B,D,1,0.50,0.333333
8,B,E,0,0.75,1.000000
9,B,E,1,0.50,0.000000


### Holdout Targer Encoding

次に、Holdout Target Encoding（Holdout Target Statistic）です。

この方法は訓練データを分割し、別ブロックの目的変数の平均を使用して自身のブロックのカテゴリカル変数を変換します。

本稿で紹介するTarget Encodingの中ではもっとも一般的に知られている方法です。

絶対ではありませんが、この方法を使用するとリークを起こりにくくすることができます。

図6は図特徴量の1つに焦点を当てた時のHoldout Target Encodingです。

![](./img/image6.png)

